<img style="float: left;" src="https://cdn.pixabay.com/photo/2016/12/07/09/45/dna-1889085__340.jpg" width=10%> <h1> Application of AI to Discover Novel Binding of Small Molecules </h1>

---------
### Sample Dataset for Testing Purposes

##### Here we create a sample dataset for two reasons:
- to get a better understanding of the structure of the data
- test any sample code for validity

##### Structure of sample dataset:
1. A dataframe consisting of 50 genes and 1020 profiles [50 x 1020]
2. Columns are a combination of drug, replicate, time, concentration, probe_location, cell type. For the purposes of this project only drug and replicate matters in terms of training. So the column name will be structured as
"*drug + replicate id + unique characters that represent time, concentration, probe_location and cell type*"
3. 20 columns consist of control genes or 'control probes'. Columns are labelled control_x where x is a number from 1 to 20
3. Dataset consists of 25 drugs with 4 replicates and 10 combinations of time, concentration, probe_location and cell type

| Feature      | Quantity | Represented By |
| ----------- | ----------- | ----------- |
| Drug      | 25       | Alphabets A-Y |
| Replicate   | 4        | Numbers 1-4 |
| Other features   | 10        | Random String of length 3 |

***R_3_xcv*** represents a profile of drug 'R', of replicate 3, with other features coresponding to 'xcv'

##### Construction of Sample Dataset

In [2]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
genes = ['gene'+str(a) for a in range(50)]
drugs = [chr(a) for a in range(65, 90)]
replicates = [str(a) for a in range(1, 5)]
other_features = set()

while len(other_features)!=10:
    rand_string = "". join([str(chr(int(random.random()*100)%26+97)) for a in range(3)])
    other_features.add(rand_string)

In [4]:
columns = ["_".join([a,b,c]) for a in drugs for b in replicates for c in other_features]
# columns = ["control_"+str(a+1) for a in range(20)] + columns

In [5]:
data = pd.DataFrame(2*np.random.rand(50, len(columns))-1, index=genes, columns=columns)
data.columns = columns
data.fillna(random.random(), inplace = True)
data.shape

(50, 1000)

In [6]:
data.head()

,A_1_dbb,A_1_cgt,A_1_bpt,A_1_avv,A_1_mux,A_1_fbu,A_1_anr,A_1_beg,A_1_muj,A_1_hbp,...,Y_4_dbb,Y_4_cgt,Y_4_bpt,Y_4_avv,Y_4_mux,Y_4_fbu,Y_4_anr,Y_4_beg,Y_4_muj,Y_4_hbp
gene0,0.440710,0.444158,-0.220031,0.973719,-0.450584,-0.493187,-0.083021,0.698780,-0.813887,-0.051977,...,0.045355,-0.633882,0.061390,0.923311,0.207054,0.496728,-0.131733,-0.964608,-0.734616,-0.432657
gene1,0.976897,-0.493221,0.921762,0.320905,0.028497,-0.717272,-0.617899,0.858273,-0.799333,0.863073,...,0.694228,-0.091909,-0.021890,-0.535862,0.271671,0.595201,-0.188626,0.673659,0.963604,0.338017
gene2,-0.480933,-0.220476,0.928250,-0.450208,0.580676,-0.411313,-0.585183,-0.417979,0.333265,0.437573,...,-0.885659,0.126132,0.766619,0.133936,0.460041,-0.772729,-0.926582,0.239706,-0.627545,-0.699329
gene3,-0.254737,0.335450,-0.393548,0.927484,0.625574,-0.111955,-0.885628,0.260611,0.495056,0.135673,...,-0.681338,-0.627020,0.988549,0.710723,0.332343,0.898555,0.881959,0.088894,0.208354,-0.880364
gene4,-0.983800,-0.518821,-0.059229,0.860908,-0.019927,-0.087034,-0.482536,-0.785084,-0.372490,0.271197,...,-0.385496,-0.049900,0.198345,-0.019423,0.776551,-0.785066,-0.710670,-0.456710,-0.145681,0.590926


##### Classifying Columns
A label needs to be assigned to each class. This can be done at the biological replicate level or the perturbagen level. We create classifications for each of these.

In [7]:
perturbagen_class = [int(a/25) for a in range(1000)]
replicate_class = [10*a+c for a in range(25) for b in range(4) for c in range(10)]

##### Creating the dataset

In [8]:
#transpose data
workingdata = data.transpose()
workingdata.head()

,gene0,gene1,gene2,gene3,gene4,gene5,gene6,gene7,gene8,gene9,...,gene40,gene41,gene42,gene43,gene44,gene45,gene46,gene47,gene48,gene49
A_1_dbb,0.440710,0.976897,-0.480933,-0.254737,-0.983800,0.706868,0.009211,-0.157628,0.660318,-0.601948,...,-0.092823,-0.663170,0.335143,0.195027,-0.972612,0.609506,-0.777294,0.424781,-0.061883,0.892715
A_1_cgt,0.444158,-0.493221,-0.220476,0.335450,-0.518821,0.182545,-0.851245,0.129705,-0.333611,-0.361482,...,-0.501186,-0.270230,0.543048,-0.219838,-0.788860,0.895124,-0.983512,0.436450,-0.845838,0.523177
A_1_bpt,-0.220031,0.921762,0.928250,-0.393548,-0.059229,0.151271,0.536261,0.461935,0.542208,0.621289,...,0.908387,0.175987,0.222970,0.956481,-0.042665,0.405894,0.544508,0.931548,0.995478,0.573525
A_1_avv,0.973719,0.320905,-0.450208,0.927484,0.860908,0.893228,-0.299701,-0.260783,0.129324,-0.104638,...,0.191242,0.198645,0.341842,0.813532,0.270102,-0.746235,-0.535851,-0.230273,0.476474,0.158518
A_1_mux,-0.450584,0.028497,0.580676,0.625574,-0.019927,-0.966419,0.334955,-0.822107,0.218798,0.547418,...,-0.027106,-0.878234,-0.110892,-0.959474,-0.708271,-0.345732,-0.693933,-0.345778,0.776175,0.569883


In [9]:
X_train, X_test, y_train, y_test = train_test_split(workingdata, perturbagen_class, test_size=0.5)
X_test.shape

(500, 50)

##### Computation - Siamese

In [10]:
import keras
from keras.datasets import reuters
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers.noise import AlphaDropout
from keras.preprocessing.text import Tokenizer
from keras.layers import Layer
from tensorflow.python.keras import backend as K

/home/kavya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
max_words = 50
batch_size = 16
epochs = 40

def create_network(n_dense=6,
                   dense_units=16,
                   activation='selu',
                   dropout=AlphaDropout,
                   dropout_rate=0.1,
                   kernel_initializer='lecun_normal',
                   optimizer='adam',
                   num_classes=1,
                   max_words=max_words):
    
    model = Sequential()
    model.add(Dense(dense_units, input_shape=(max_words,),
                    kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(dropout(dropout_rate))

    for i in range(n_dense - 1):
        model.add(Dense(dense_units, kernel_initializer=kernel_initializer))
        model.add(Activation(activation))
        model.add(dropout(dropout_rate))

    #model.add(Dense(num_classes))
    #model.add(Activation('softmax'))
    return model

In [12]:
network = {
    'n_dense': 6,
    'dense_units': 16,
    'activation': 'selu',
    'dropout': AlphaDropout,
    'dropout_rate': 0.1,
    'kernel_initializer': 'lecun_normal',
    'optimizer': 'sgd',
    'num_classes':40
}

In [13]:
model = create_network(**network)

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
class ManDist(Layer):
    
    # initialize the layer, No need to include inputs parameter!
    def __init__(self, **kwargs):
        self.result = None
        super(ManDist, self).__init__(**kwargs)

    # input_shape will automatic collect input shapes to build layer
    def build(self, input_shape):
        super(ManDist, self).build(input_shape)

    # This is where the layer's logic lives.
    def call(self, x, **kwargs):
        self.result = K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True)
        return self.result

    # return output shape
    def compute_output_shape(self, input_shape):
        return K.int_shape(self.result)

In [15]:
left_input = Input(shape=(max_words,))
right_input = Input(shape=(max_words,))

In [16]:
# Model variables
shared_model = model

In [17]:
#TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'- embedding layer is required
#Node error -> from keras not from tf.python.keras
#Input 'b' of 'MatMul' Op has type float32 that does not match type int32 of argument 'a'. ->
malstm_distance = ManDist()([shared_model(left_input), shared_model(right_input)])
model = Model(inputs=[left_input, right_input], outputs=[malstm_distance])

In [18]:
model.compile(loss='mean_squared_error', optimizer="adam", metrics=['accuracy'])
model.summary()
shared_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 16)           2176        input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
man_dist_1 (ManDist)            (None, 1)            0           sequential_1[1][0]               
          

In [19]:
#ValueError: Error when checking target: expected man_dist_1 to have shape (1,) but got array with shape (46,)
#==> need to convert code to suit multi-class

malstm_trained = model.fit([X_train,X_test], y_train, epochs=10, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
500/500 [==============================] - 3s 6ms/step - loss: 147.6061 - acc: 0.0240
Epoch 2/10
500/500 [==============================] - 0s 117us/step - loss: 145.8876 - acc: 0.0220
Epoch 3/10
500/500 [==============================] - 0s 96us/step - loss: 161.2303 - acc: 0.0140
Epoch 4/10
500/500 [==============================] - 0s 107us/step - loss: 153.4704 - acc: 0.0240
Epoch 5/10
500/500 [==============================] - 0s 98us/step - loss: 142.6647 - acc: 0.0220
Epoch 6/10
500/500 [==============================] - 0s 101us/step - loss: 158.0525 - acc: 0.0240
Epoch 7/10
500/500 [==============================] - 0s 104us/step - loss: 152.8522 - acc: 0.0220
Epoch 8/10
500/500 [==============================] - 0s 106us/step - loss: 152.9358 - acc: 0.0160
Epoch 9/10
500/500 [==============================] - 0s 124us/step - loss: 146.8447 - acc: 0.0220
Epoch 10/10
500/500 [==============================] - 0s 134us/s

In [20]:
prediction = model.predict([X_test,X_train],verbose=1)
print(prediction[0:5])

500/500 [==============================] - 0s 669us/step
[[15.165006 ]
 [16.83998  ]
 [ 9.79505  ]
 [ 5.4231663]
 [16.950424 ]]


In [21]:
score = model.evaluate([X_test,X_train],y_train,verbose=1)
score

500/500 [==============================] - 0s 538us/step


[158.13666235351562, 0.016]

##### Normal Neural Network

In [63]:
one_hot_perturbagen = np.zeros((1000, 40))
for a in range(len(perturbagen_class)):
    one_hot_perturbagen[a][perturbagen_class[a]] = 1
one_hot_perturbagen


array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [80]:
network = {
    'n_dense': 6,
    'dense_units': 40,
    'activation': 'selu',
    'dropout': AlphaDropout,
    'dropout_rate': 0.1,
    'kernel_initializer': 'lecun_normal',
    'optimizer': 'sgd',
    'num_classes':40
}

model = create_network(**network)
model.add(Dense(40))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_102 (Dense)            (None, 40)                2040      
_________________________________________________________________
activation_99 (Activation)   (None, 40)                0         
_________________________________________________________________
alpha_dropout_91 (AlphaDropo (None, 40)                0         
_________________________________________________________________
dense_103 (Dense)            (None, 40)                1640      
_________________________________________________________________
activation_100 (Activation)  (None, 40)                0         
_________________________________________________________________
alpha_dropout_92 (AlphaDropo (None, 40)                0         
_________________________________________________________________
dense_104 (Dense)            (None, 40)                1640      
__________

In [81]:
model.predict(np.random.rand(1, 50))
# np.random.rand(50).shape

array([[0.02942943, 0.02565352, 0.00144317, 0.00974138, 0.01525019,
        0.0104328 , 0.07446111, 0.02203848, 0.01232203, 0.0542004 ,
        0.01470957, 0.04203646, 0.01324992, 0.08094575, 0.01636207,
        0.04082963, 0.03705608, 0.00441103, 0.00109657, 0.02596479,
        0.0568582 , 0.01609403, 0.01140684, 0.01454322, 0.03377102,
        0.00824484, 0.00944351, 0.13910936, 0.00302836, 0.00667706,
        0.01429386, 0.02198435, 0.01339544, 0.01633186, 0.00851826,
        0.01925597, 0.02709278, 0.01147179, 0.00713861, 0.02970624]],
      dtype=float32)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(workingdata, one_hot_perturbagen, test_size=0.2)

In [85]:
model.fit([X_train], y_train, epochs=1000, verbose=1)

Epoch 1/1000
800/800 [==============================] - 0s 129us/step - loss: 3.1402 - acc: 0.1338
Epoch 2/1000
800/800 [==============================] - 0s 101us/step - loss: 3.0734 - acc: 0.1500
Epoch 3/1000
800/800 [==============================] - 0s 131us/step - loss: 3.0980 - acc: 0.1475
Epoch 4/1000
800/800 [==============================] - 0s 155us/step - loss: 3.0704 - acc: 0.1463
Epoch 5/1000
800/800 [==============================] - 0s 116us/step - loss: 3.0907 - acc: 0.1537
Epoch 6/1000
800/800 [==============================] - 0s 136us/step - loss: 3.0924 - acc: 0.1537
Epoch 7/1000
800/800 [==============================] - 0s 114us/step - loss: 3.0609 - acc: 0.1487
Epoch 8/1000
800/800 [==============================] - 0s 147us/step - loss: 3.1072 - acc: 0.1613
Epoch 9/1000
800/800 [==============================] - 0s 119us/step - loss: 3.0742 - acc: 0.1600
Epoch 10/1000
800/800 [==============================] - 0s 165us/step - loss: 3.0030 - acc: 0.1625
Epoch 11/

800/800 [==============================] - 0s 185us/step - loss: 2.6971 - acc: 0.2475
Epoch 84/1000
800/800 [==============================] - 0s 158us/step - loss: 2.6834 - acc: 0.2225
Epoch 85/1000
800/800 [==============================] - 0s 123us/step - loss: 2.6815 - acc: 0.2238
Epoch 86/1000
800/800 [==============================] - 0s 105us/step - loss: 2.6836 - acc: 0.2300
Epoch 87/1000
800/800 [==============================] - 0s 140us/step - loss: 2.6877 - acc: 0.2325
Epoch 88/1000
800/800 [==============================] - 0s 138us/step - loss: 2.7320 - acc: 0.2363
Epoch 89/1000
800/800 [==============================] - 0s 150us/step - loss: 2.7037 - acc: 0.2337
Epoch 90/1000
800/800 [==============================] - 0s 154us/step - loss: 2.6475 - acc: 0.2400
Epoch 91/1000
800/800 [==============================] - 0s 149us/step - loss: 2.6829 - acc: 0.2287
Epoch 92/1000
800/800 [==============================] - 0s 134us/step - loss: 2.7167 - acc: 0.2200
Epoch 93/1000


800/800 [==============================] - 0s 167us/step - loss: 2.4922 - acc: 0.2725
Epoch 165/1000
800/800 [==============================] - 0s 101us/step - loss: 2.4627 - acc: 0.2762
Epoch 166/1000
800/800 [==============================] - 0s 104us/step - loss: 2.4696 - acc: 0.2888
Epoch 167/1000
800/800 [==============================] - 0s 110us/step - loss: 2.5609 - acc: 0.2613
Epoch 168/1000
800/800 [==============================] - 0s 133us/step - loss: 2.4399 - acc: 0.2863
Epoch 169/1000
800/800 [==============================] - 0s 133us/step - loss: 2.4842 - acc: 0.2725
Epoch 170/1000
800/800 [==============================] - 0s 143us/step - loss: 2.4928 - acc: 0.2725
Epoch 171/1000
800/800 [==============================] - 0s 172us/step - loss: 2.3872 - acc: 0.2825
Epoch 172/1000
800/800 [==============================] - 0s 129us/step - loss: 2.4422 - acc: 0.2775
Epoch 173/1000
800/800 [==============================] - 0s 97us/step - loss: 2.4400 - acc: 0.2850
Epoch 

800/800 [==============================] - 0s 164us/step - loss: 2.3160 - acc: 0.3075
Epoch 246/1000
800/800 [==============================] - 0s 98us/step - loss: 2.3019 - acc: 0.3313
Epoch 247/1000
800/800 [==============================] - 0s 149us/step - loss: 2.2583 - acc: 0.3150
Epoch 248/1000
800/800 [==============================] - 0s 139us/step - loss: 2.3489 - acc: 0.3225
Epoch 249/1000
800/800 [==============================] - 0s 104us/step - loss: 2.2835 - acc: 0.3112
Epoch 250/1000
800/800 [==============================] - 0s 237us/step - loss: 2.2711 - acc: 0.3475
Epoch 251/1000
800/800 [==============================] - 0s 185us/step - loss: 2.2728 - acc: 0.3550
Epoch 252/1000
800/800 [==============================] - 0s 157us/step - loss: 2.2783 - acc: 0.3362
Epoch 253/1000
800/800 [==============================] - 0s 201us/step - loss: 2.3294 - acc: 0.3025
Epoch 254/1000
800/800 [==============================] - 0s 192us/step - loss: 2.3155 - acc: 0.3175
Epoch 

800/800 [==============================] - 0s 244us/step - loss: 2.1253 - acc: 0.3488
Epoch 327/1000
800/800 [==============================] - 0s 146us/step - loss: 2.1040 - acc: 0.3763
Epoch 328/1000
800/800 [==============================] - 0s 149us/step - loss: 2.1415 - acc: 0.3513
Epoch 329/1000
800/800 [==============================] - 0s 114us/step - loss: 2.1155 - acc: 0.3613
Epoch 330/1000
800/800 [==============================] - 0s 197us/step - loss: 2.1795 - acc: 0.3562
Epoch 331/1000
800/800 [==============================] - 0s 153us/step - loss: 2.1915 - acc: 0.3513
Epoch 332/1000
800/800 [==============================] - 0s 111us/step - loss: 2.1635 - acc: 0.3837
Epoch 333/1000
800/800 [==============================] - 0s 154us/step - loss: 2.1594 - acc: 0.3362
Epoch 334/1000
800/800 [==============================] - 0s 129us/step - loss: 2.1887 - acc: 0.3500
Epoch 335/1000
800/800 [==============================] - 0s 169us/step - loss: 2.1359 - acc: 0.3650
Epoch

...

800/800 [==============================] - 0s 155us/step - loss: 1.8828 - acc: 0.4550
Epoch 651/1000
800/800 [==============================] - 0s 102us/step - loss: 1.9166 - acc: 0.4437
Epoch 652/1000
800/800 [==============================] - 0s 112us/step - loss: 1.8579 - acc: 0.4475
Epoch 653/1000
800/800 [==============================] - 0s 106us/step - loss: 1.8042 - acc: 0.4725
Epoch 654/1000
800/800 [==============================] - 0s 110us/step - loss: 1.6531 - acc: 0.5075
Epoch 655/1000
800/800 [==============================] - 0s 132us/step - loss: 1.8088 - acc: 0.4575
Epoch 656/1000
800/800 [==============================] - 0s 136us/step - loss: 1.7559 - acc: 0.4550
Epoch 657/1000
800/800 [==============================] - 0s 131us/step - loss: 1.7820 - acc: 0.4713
Epoch 658/1000
800/800 [==============================] - 0s 129us/step - loss: 1.9561 - acc: 0.4413
Epoch 659/1000
800/800 [==============================] - 0s 97us/step - loss: 1.8597 - acc: 0.4563
Epoch 

800/800 [==============================] - 0s 200us/step - loss: 1.8226 - acc: 0.4400
Epoch 732/1000
800/800 [==============================] - 0s 258us/step - loss: 1.8197 - acc: 0.4525
Epoch 733/1000
800/800 [==============================] - 0s 214us/step - loss: 1.7911 - acc: 0.4662
Epoch 734/1000
800/800 [==============================] - 0s 120us/step - loss: 1.7758 - acc: 0.4688
Epoch 735/1000
800/800 [==============================] - 0s 121us/step - loss: 1.7456 - acc: 0.4812
Epoch 736/1000
800/800 [==============================] - 0s 129us/step - loss: 1.8197 - acc: 0.4650
Epoch 737/1000
800/800 [==============================] - 0s 109us/step - loss: 1.8902 - acc: 0.4262
Epoch 738/1000
800/800 [==============================] - 0s 159us/step - loss: 1.8184 - acc: 0.4688
Epoch 739/1000
800/800 [==============================] - 0s 103us/step - loss: 1.7736 - acc: 0.4887
Epoch 740/1000
800/800 [==============================] - 0s 226us/step - loss: 1.8320 - acc: 0.4525
Epoch

800/800 [==============================] - 0s 132us/step - loss: 1.8302 - acc: 0.4625
Epoch 813/1000
800/800 [==============================] - 0s 118us/step - loss: 1.7864 - acc: 0.4388
Epoch 814/1000
800/800 [==============================] - 0s 157us/step - loss: 1.6921 - acc: 0.4875
Epoch 815/1000
800/800 [==============================] - 0s 162us/step - loss: 1.7432 - acc: 0.4763
Epoch 816/1000
800/800 [==============================] - 0s 108us/step - loss: 1.8550 - acc: 0.4637
Epoch 817/1000
800/800 [==============================] - 0s 124us/step - loss: 1.8164 - acc: 0.4625
Epoch 818/1000
800/800 [==============================] - 0s 123us/step - loss: 1.7831 - acc: 0.4713
Epoch 819/1000
800/800 [==============================] - 0s 145us/step - loss: 1.7797 - acc: 0.4550
Epoch 820/1000
800/800 [==============================] - 0s 132us/step - loss: 1.6826 - acc: 0.4925
Epoch 821/1000
800/800 [==============================] - 0s 126us/step - loss: 1.7705 - acc: 0.4850
Epoch

800/800 [==============================] - 0s 154us/step - loss: 1.6866 - acc: 0.4913
Epoch 893/1000
800/800 [==============================] - 0s 124us/step - loss: 1.6547 - acc: 0.5237
Epoch 894/1000
800/800 [==============================] - 0s 97us/step - loss: 1.7234 - acc: 0.5025
Epoch 895/1000
800/800 [==============================] - 0s 109us/step - loss: 1.7102 - acc: 0.4925
Epoch 896/1000
800/800 [==============================] - 0s 142us/step - loss: 1.6741 - acc: 0.5087
Epoch 897/1000
800/800 [==============================] - 0s 107us/step - loss: 1.7441 - acc: 0.4850
Epoch 898/1000
800/800 [==============================] - 0s 100us/step - loss: 1.8267 - acc: 0.4725
Epoch 899/1000
800/800 [==============================] - 0s 123us/step - loss: 1.7496 - acc: 0.4550
Epoch 900/1000
800/800 [==============================] - 0s 103us/step - loss: 1.6907 - acc: 0.4975
Epoch 901/1000
800/800 [==============================] - 0s 103us/step - loss: 1.7133 - acc: 0.4875
Epoch 

800/800 [==============================] - 0s 209us/step - loss: 1.6469 - acc: 0.5038
Epoch 974/1000
800/800 [==============================] - 0s 166us/step - loss: 1.7046 - acc: 0.5200
Epoch 975/1000
800/800 [==============================] - 0s 179us/step - loss: 1.7313 - acc: 0.4725
Epoch 976/1000
800/800 [==============================] - 0s 143us/step - loss: 1.7219 - acc: 0.5012
Epoch 977/1000
800/800 [==============================] - 0s 124us/step - loss: 1.7274 - acc: 0.4925
Epoch 978/1000
800/800 [==============================] - 0s 135us/step - loss: 1.6983 - acc: 0.5025
Epoch 979/1000
800/800 [==============================] - 0s 141us/step - loss: 1.7464 - acc: 0.4825
Epoch 980/1000
800/800 [==============================] - 0s 162us/step - loss: 1.7288 - acc: 0.4887
Epoch 981/1000
800/800 [==============================] - 0s 176us/step - loss: 1.6550 - acc: 0.5212
Epoch 982/1000
800/800 [==============================] - 0s 129us/step - loss: 1.6076 - acc: 0.5250
Epoch

In [87]:
model.evaluate(x=X_test, y=y_test)
model.evaluate(x=X_train, y=y_train)

800/800 [==============================] - 0s 68us/step


[0.30319243371486665, 0.91375]